In [1]:

import os, sys
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

from utils import instantiate_from_cfg, save_files
from utils.util import transform_date_str, set_seed
import utils.misc as misc


In [4]:
from models.vit.mae_gaze import MAE_Gaze

mae_b16 = MAE_Gaze(model_type='vit_b_16', custom_pretrained_path='checkpoints/mae_b16/mae_b16_checkpoint-299.pth')
mae_b16 = MAE_Gaze(model_type='vit_l_16', custom_pretrained_path='checkpoints/mae_l16/mae_l16_checkpoint-299.pth')
mae_h14 = MAE_Gaze(model_type='vit_h_14', custom_pretrained_path='checkpoints/mae_h14/mae_h14_checkpoint-299.pth')


Loaded custom pretrained weights from checkpoints/mae_b16/mae_b16_checkpoint-299.pth
Loaded custom pretrained weights from checkpoints/mae_l16/mae_l16_checkpoint-299.pth


In [7]:
mae_h14_crossX = '/home/jqin/wk/TUD/UniGaze/unigaze/logs/mae_H14_cross_X_onecycle_6epochs/20-30-35_seed42_3a8c7a62_rank0/ckpt/epoch_06.pth.tar'


model_state = torch.load(mae_h14_crossX, map_location='cpu')['model_state']
if next(iter(model_state.keys())).startswith('module.'):
    model_state = dict([(k[7:], v) for k, v in model_state.items()])

print(model_state.keys())
mae_h14.load_state_dict(model_state, strict=True)

odict_keys(['vit.cls_token', 'vit.pos_embed', 'vit.patch_embed.proj.weight', 'vit.patch_embed.proj.bias', 'vit.blocks.0.norm1.weight', 'vit.blocks.0.norm1.bias', 'vit.blocks.0.attn.qkv.weight', 'vit.blocks.0.attn.qkv.bias', 'vit.blocks.0.attn.proj.weight', 'vit.blocks.0.attn.proj.bias', 'vit.blocks.0.norm2.weight', 'vit.blocks.0.norm2.bias', 'vit.blocks.0.mlp.fc1.weight', 'vit.blocks.0.mlp.fc1.bias', 'vit.blocks.0.mlp.fc2.weight', 'vit.blocks.0.mlp.fc2.bias', 'vit.blocks.1.norm1.weight', 'vit.blocks.1.norm1.bias', 'vit.blocks.1.attn.qkv.weight', 'vit.blocks.1.attn.qkv.bias', 'vit.blocks.1.attn.proj.weight', 'vit.blocks.1.attn.proj.bias', 'vit.blocks.1.norm2.weight', 'vit.blocks.1.norm2.bias', 'vit.blocks.1.mlp.fc1.weight', 'vit.blocks.1.mlp.fc1.bias', 'vit.blocks.1.mlp.fc2.weight', 'vit.blocks.1.mlp.fc2.bias', 'vit.blocks.2.norm1.weight', 'vit.blocks.2.norm1.bias', 'vit.blocks.2.attn.qkv.weight', 'vit.blocks.2.attn.qkv.bias', 'vit.blocks.2.attn.proj.weight', 'vit.blocks.2.attn.proj.bia

<All keys matched successfully>

In [13]:
def clean_ckpt(ckpt_input, ckpt_output):
    model_state = torch.load(ckpt_input, map_location='cpu')['model_state']
    if next(iter(model_state.keys())).startswith('module.'):
        model_state = dict([(k[7:], v) for k, v in model_state.items()])
    torch.save({'model_state':model_state},
                ckpt_output)

In [8]:
unigaze_b16 = MAE_Gaze(model_type='vit_b_16')